<a href="https://colab.research.google.com/github/ram130849/Deep_Learning_Systems_Assignments/blob/main/TensorFlow/Sushant/DLS_Assignment4_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import timeit
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Flatten
from keras.models import Model
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

### Allocating and Checking the GPU

In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
0.41343161400001804
GPU (s):
0.039932116000017004
GPU speedup over CPU: 10x


### Loading the MNIST Dataset

In [4]:
mnist = tf.keras.datasets.mnist

In [5]:
(train_images, train_labels) , (test_images, test_labels) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [6]:
#Lets look at the training data
print("Training Images Shape: ",train_images.shape)
print("Training Labels: ",train_labels)

#Lets look at the testing data
print("Testing Images Shape: ",test_images.shape)
print("Testing Labels: ",test_labels)

Training Images Shape:  (60000, 28, 28)
Training Labels:  [5 0 4 ... 5 6 8]
Testing Images Shape:  (10000, 28, 28)
Testing Labels:  [7 2 1 ... 4 5 6]


### Creating the Baseline Model

In [17]:
baseline = keras.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(1024, activation='relu', name='first_layer',kernel_initializer=keras.initializers.GlorotNormal()),
    layers.Dense(1024, activation='relu', name='second_layer',kernel_initializer=keras.initializers.GlorotNormal()),
    layers.Dense(1024, activation='relu', name='third_layer',kernel_initializer=keras.initializers.GlorotNormal()),
    layers.Dense(1024, activation='relu', name='fourth_layer',kernel_initializer=keras.initializers.GlorotNormal()),
    layers.Dense(1024, activation='relu', name="fifth_layser",kernel_initializer=keras.initializers.GlorotNormal()),
    layers.Dense(10, activation='softmax')
])
print(baseline.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 first_layer (Dense)         (None, 1024)              803840    
                                                                 
 second_layer (Dense)        (None, 1024)              1049600   
                                                                 
 third_layer (Dense)         (None, 1024)              1049600   
                                                                 
 fourth_layer (Dense)        (None, 1024)              1049600   
                                                                 
 fifth_layser (Dense)        (None, 1024)              1049600   
                                                                 
 dense_2 (Dense)             (None, 10)               

In [18]:
baseline.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

### Training the Baseline Model

In [19]:
history_baseline = baseline.fit(train_images , train_labels ,epochs = 30, batch_size = 256)

Epoch 1/30


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


235/235 [==============================] - 1s 3ms/step - loss: 3.3367 - sparse_categorical_accuracy: 0.8655
Epoch 2/30
235/235 [==============================] - 1s 3ms/step - loss: 0.1149 - sparse_categorical_accuracy: 0.9647
Epoch 3/30
235/235 [==============================] - 1s 3ms/step - loss: 0.0728 - sparse_categorical_accuracy: 0.9768
Epoch 4/30
235/235 [==============================] - 1s 3ms/step - loss: 0.0583 - sparse_categorical_accuracy: 0.9811
Epoch 5/30
235/235 [==============================] - 1s 3ms/step - loss: 0.0528 - sparse_categorical_accuracy: 0.9835
Epoch 6/30
235/235 [==============================] - 1s 3ms/step - loss: 0.0459 - sparse_categorical_accuracy: 0.9848
Epoch 7/30
235/235 [==============================] - 1s 3ms/step - loss: 0.0408 - sparse_categorical_accuracy: 0.9872
Epoch 8/30
235/235 [==============================] - 1s 3ms/step - loss: 0.0325 - sparse_categorical_accuracy: 0.9893
Epoch 9/30
235/235 [==============================] - 1s 3m

In [20]:
evaluation = baseline.evaluate(test_images , test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 0.1063 - sparse_categorical_accuracy: 0.9805


### Saving the Baseline Model

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
model_path = '/content/drive/MyDrive/DLS_Assignments/Models/assign4_part1_baseline.h5'
baseline.save(model_path)